In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dateutil.parser import parse
from time import sleep, time
from random import randint
import re
from IPython.core.display import clear_output
from warnings import warn
import numpy as np
from selenium import webdriver 
import cv2 
import pytesseract
from PIL import Image

In [2]:
def is_captcha_page(soup):
    captcha_text="Please verify you are human by typing the text below and clicking submit."
    page_info=soup.find('font',attrs={'class':"pagebodybold"})
    if page_info!=None and captcha_text in page_info.text:
        return True
    return False

In [3]:
def solve_captcha(company_link):
    driver = webdriver.Firefox('/home/udit/projects/chromiumm')
    driver.get(company_link)
    element = driver.find_element_by_id("imgCaptcha")
    element.screenshot('final.png')
    img = cv2.imread('final.png')
    captcha=pytesseract.image_to_string(img)
    
    captcha=re.sub(r'\W+', '', captcha)
    captcha_input = driver.find_element_by_xpath("//input[@id ='txtCaptcha']") 
    captcha_input.send_keys(captcha) 

    # submit button clicked
    driver.find_element_by_xpath("//input[@id ='btnRegiser']").click()
    html = driver.page_source
    soup = BeautifulSoup(html)
    driver.close()
    return soup

In [4]:
def try_filling(strr,company_info):
    try:
        global x
        eqn= "{0} = {1}".format("x",strr)
        x = np.nan
        exec(eqn,globals())
        return x
    except IndexError:
        return(np.nan)

In [5]:
URL = "https://ibanknet.com/scripts/callreports/fiList.aspx?type=fry9"
r = requests.get(URL)
soup = BeautifulSoup(r.content, 'html5lib')

In [6]:
companies=soup.find_all('a', attrs = {'class':'pagebody'}) 

In [7]:
all_companies=[]
error_links=[]

for company in companies:
    company_link="https://ibanknet.com/"+company.get('href')
    try:
        c = requests.get(company_link)
        soup = BeautifulSoup(c.content, 'html5lib')

        while(is_captcha_page(soup)):
            soup=solve_captcha(company_link)

        company_info=[]

        for info in soup.find_all('td', attrs = {'class':'instinfo'}):
            company_info.append(info.text)


        company_inf={}
        company_inf["company_name"]=soup.find('font',attrs={'class':"blockuhilite"}).text.strip()
        company_inf["RSSD_ID"]=company_info[0].split()[2]
        company_inf['RefinedAddress']=company_info[1]
        company_inf['City']=company_info[4].split(",")[0].strip()
        company_inf['State']=company_info[4].split(",")[1].strip()
        company_inf['Pin']=company_info[4].split(",")[2].strip()
        company_inf['YearOpened']=company_info[company_info.index('Year Opened : ')+1]
        company_inf['ReserveDistrict']=company_info[company_info.index('Reserve District : ')+1]
        company_inf['EntityType']=company_info[company_info.index('Entity Type : ')+1]
        company_inf['Structure']=company_info[company_info.index('Structure : ')+1]
        company_inf['InstitutionType']=company_info[company_info.index('Institution Type : ')+1]
        company_inf['FederalRegulator']=company_info[company_info.index('Federal Regulator : ')+1]
        company_inf['S-Corp']=company_info[company_info.index('S-Corp : ')+1]
        all_companies.append(company_inf)
        
    except Exception as e:
        error_links.append([company_link,str(e)])

In [8]:
all_compaies=pd.DataFrame(all_companies)

In [9]:
all_compaies.head()

,company_name,RSSD_ID,RefinedAddress,City,State,Pin,YearOpened,ReserveDistrict,EntityType,Structure,InstitutionType,FederalRegulator,S-Corp
0,JPMORGAN CHASE & CO.,1039502,383 MADISON AVENUE,NEW YORK,NY,10179,1968,New York,Financial Holding Company - Domestic,Corporation (stock),Holding Company,FED,No
1,BANK OF AMERICA CORPORATION,1073757,100 NORTH TRYON STREET,CHARLOTTE,NC,28255,1968,Richmond,Financial Holding Company - Domestic,Corporation (stock),Holding Company,FED,No
2,CITIGROUP INC.,1951350,388 GREENWICH STREET,NEW YORK,NY,10013,1912,New York,Financial Holding Company - Domestic,Corporation (stock),Holding Company,FED,No
3,WELLS FARGO & COMPANY,1120754,420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1929,San Francisco,Financial Holding Company - Domestic,Corporation (stock),Holding Company,FED,No
4,"GOLDMAN SACHS GROUP, INC., THE",2380443,200 WEST STREET,NEW YORK,NY,10282,1990,New York,Financial Holding Company - Domestic,Corporation (stock),Securities broker and/or dealer,FED,No


In [10]:
error_links

[]

In [11]:
#all_compaies.to_csv("ibankFry9.csv",index=False)